In [258]:
import pandas as pd
import numpy as np
import operator
import pprint
tree={}

In [315]:
def entropy(class_distribution):
    return np.sum(np.log(1/class_distribution) * class_distribution)

In [561]:
def information_gain(entropy_levelbefore, entropy_oflevel, data_points_in_each_node):
    return entropy_levelbefore - (np.sum((entropy_oflevel*data_points_in_each_node)/np.sum(data_points_in_each_node)))

In [422]:
def find_subtree(dataset,feature,side):
    if side == 'left':
        f= feature.split(" ")
        l= dataset[dataset[f[0]] >= float(f[1])]
        return l
    else:
        f= feature.split(" ")
        l= dataset[dataset[f[0]] < float(f[1])]
        return l

In [433]:
def select_feature(dataset,prev_entropy,entropy_cache):
    en={}
    info={}
    cols=dataset.columns
    cols=list(cols)
    del cols[-1]
    
    for feature in cols:
        for s in set(list(dataset[feature])):
                left= dataset[dataset[feature] >= s]
                right=dataset[dataset[feature] < s]
                
                left_distribution =(np.asarray(list(left['class'].value_counts())))/len(left['class'])
                right_distribution =(np.asarray(list(right['class'].value_counts())))/len(right['class'])
                
                if str(left_distribution) not in entropy_cache:
                    en[feature+" left"] = entropy(left_distribution)
                    entropy_cache[str(left_distribution)]=en[feature+" left"]
                else:
                    en[feature+" left"] =entropy_cache[str(left_distribution)]

                if str(right_distribution) not in entropy_cache:
                    en[feature+" right"] = entropy(right_distribution)
                    entropy_cache[str(right_distribution)]=en[feature+" right"]
                else:
                    en[feature+" right"]=entropy_cache[str(right_distribution)]
               
                info[str(feature)+" "+str(s)]=information_gain(prev_entropy,np.asarray([float(en[feature+" left"]),float(en[feature+" right"])]),np.asarray([len(left),len(right)]))
    best_feature=max(info.items(), key=operator.itemgetter(1))[0]
    return best_feature

In [583]:
data = open('spambase.data').read().split("\n")
data = [i.split(",") for i in (data)]
column = open('features','r').read().split("\n")
column =[i.strip() for i in column]
dataset =  pd.DataFrame(data, columns=column)
dataset=dataset.convert_objects(convert_numeric=True)
dataset=dataset.dropna()
columnmax = [max(dataset[i]) for i in dataset.columns]
columnmin = [min(dataset[i]) for i in dataset.columns]
for i,j in enumerate(dataset.columns):
    dataset[j] = (dataset[j] - columnmin[i])/columnmax[i]
dataset['class']= dataset['class'].astype(int)
training_dataset=dataset.sample(frac=1)[:-1000]
test_dataset=dataset.sample(frac=1)[-1000:]

/Users/shaivalpatel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [584]:

def training(dataset,depth,tree,entropy_cache={}):
    current_node_entropy = entropy((np.asarray(list(dataset['class'].value_counts())))/len(dataset['class']))
    if (current_node_entropy>=0.3 and depth !=0):
        
            current_node_entropy = entropy((np.asarray(list(dataset['class'].value_counts())))/len(dataset['class']))
            best_feature = select_feature(dataset,current_node_entropy,entropy_cache)
            print(best_feature)
            left_subtree= find_subtree(dataset,best_feature,"left")
            right_subtree=find_subtree(dataset,best_feature,"right")
            if not left_subtree.empty:
                tree["> "+str(best_feature)]=training(left_subtree,depth-1,{},entropy_cache)
            if not right_subtree.empty:
                tree["< "+str(best_feature)]=training(right_subtree,depth-1,{},entropy_cache)
            return tree
    else:
        return (dataset['class'].value_counts())

In [585]:
tree=training(training_dataset,10,tree)

char_freq_! 0.002432415789149578
char_freq_$ 0.0021655838747293025
capital_run_length_average 0.0015573696145124718
word_freq_hp 0.010081613058089295
word_freq_mail 0.06545654565456545
word_freq_edu 0.025396825396825397
word_freq_our 0.015
word_freq_free 0.0315
char_freq_! 0.01573372744627132
capital_run_length_total 0.00523956820907771
word_freq_re 0.05742296918767507
word_freq_remove 0.02751031636863824
capital_run_length_longest 0.0015016518169986986
capital_run_length_average 0.0018140589569160999
word_freq_remove 0.024759284731774415
char_freq_! 0.025063119650224768
capital_run_length_total 0.0008837825894829872
capital_run_length_total 0.004608294930875576
word_freq_you 0.1664
word_freq_your 0.11611161116111612
word_freq_money 0.3328
word_freq_free 0.06999999999999999
char_freq_! 0.028480817784346326
capital_run_length_total 0.00012625465564042673
word_freq_your 0.05580558055805581
word_freq_business 0.05042016806722689
word_freq_our 0.014000000000000002
char_freq_( 0.01507383100

In [516]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(tree)

{   '< char_freq_$ 0.009328668998833917': {   '< word_freq_remove 0.008253094910591473': {   '< char_freq_! 0.005911694069831886': 0    1864
1     165
Name: class, dtype: int64,
                                                                                             '> char_freq_! 0.005911694069831886': {   '< capital_run_length_average 0.0016117913832199545': {   '< word_freq_free 0.006500000000000001': {   '< word_freq_internet 0.052205220522052204': {   '< word_freq_addresses 0.04081632653061224': {   '< char_freq_! 0.02564813104255188': 0    175
1     13
Name: class, dtype: int64,
                                                                                                                                                                                                                                                                                                                                                '> char_freq_! 0.02564813104255188': {   '< word_freq_you 0.06453333

In [502]:
def testing(row,tree,depth):
 
    if  depth !=0 and type(tree)==dict :
        for key in tree.keys():
            operator = key.split(" ")[0]
            feature =key.split(" ")[1]
            threshold=key.split(" ")[2]
            row_feature = row[feature]
            if operator == '<':
                if (row_feature)< float(threshold):
                    result=testing(row,tree[key],depth-1)
                    return result
            else:
                if (row_feature)>= float(threshold):
                    result=testing(row,tree[key],depth-1)
                    return result
    else:
        
        if 0 not in tree:
            return 1
        elif 1 not in tree:
            return 0
        else:
            if tree[0]>tree[1]:
                return 0
            else:
                return 1

In [586]:
prediction=[]
for index,row in test_dataset.iterrows():
    prediction.append(testing(row,tree,10))

In [599]:
def accuracy(prediction,test_dataset):
    test=list(test_dataset['class'])
    ac = zip(prediction,test)
    right =0
    for k,v in ac:
        if k ==v:
            right+=1
    return right/len(prediction)

In [600]:
acc = accuracy(prediction,test_dataset)

In [602]:
acc

0.937